<a href="https://colab.research.google.com/github/bhimnexus/bhimnexus/blob/main/implementing_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages (quiet)
!pip install -q termcolor langchain-openai langchain openai langchain_experimental tiktoken python-docx
!pip install -q faiss-cpu==1.9.0.post1

In [ ]:
import os

# Option A - set directly (only if this notebook is private)
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Option B - interactive (safer)
from getpass import getpass
key = getpass("OpenAI API key (input will be hidden): ")
os.environ["OPENAI_API_KEY"] = key


OpenAI API key (input will be hidden): ··········


In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)

from termcolor import colored
from datetime import datetime, timedelta
import math
import os
import sys

# -------------------------
# FAISS (working version)
# -------------------------
import faiss
print("faiss version:", faiss.__version__)

# -------------------------
# LangChain v0.1.14 + OpenAI v0.0.8 imports
# -------------------------
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.vectorstores import FAISS
from langchain.schema import Document

# -------------------------
# Experimental generative agents
# (these may or may not load depending on version)
# -------------------------
try:
    from langchain_experimental.generative_agents import (
        GenerativeAgent,
        GenerativeAgentMemory,
    )
    print("GenerativeAgent import OK.")
except Exception as e:
    print("GenerativeAgent import failed:", e)

print("\nImports OK. Python version:", sys.version)


faiss version: 1.9.0
GenerativeAgent import OK.

Imports OK. Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [ ]:
# Path to your uploaded resume
file_path = "/content/Bhim_Singh_Senior COBOL_Developer_Resume.docx"
print("Loading:", file_path)

from docx import Document as DocxDocument

def load_docx(path):
    doc = DocxDocument(path)
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    return "\n\n".join(paragraphs)

raw_text = load_docx(file_path)

print("Loaded text length:", len(raw_text), "characters")
print("\n--- Resume Preview (first 1000 chars) ---\n")
print(raw_text[:1000])


Loading: /content/Bhim_Singh_Senior COBOL_Developer_Resume.docx
Loaded text length: 2808 characters

--- Resume Preview (first 1000 chars) ---

BHIM SINGH

📞 +91 9810811949 | ✉️ bhim.sb@gmail.com | 🌐 www.linkedin.com/in/bhim-singh-mainframelead

PROFILE SUMMARY

Senior Mainframe Developer with 13+ years of hands-on experience in designing, coding, testing, and maintaining large-scale Mainframe applications using COBOL, JCL, VSAM, DB2, and CICS across banking, insurance, and healthcare domains. Skilled in performance tuning, production support, batch optimization, and end-to-end SDLC delivery. Strong problem-solver with proven ability to deliver stable, high-performing mainframe systems aligned with business and SLA targets.

CORE TECHNICAL SKILLS

Languages: COBOL, JCL, PL/I, ASSEMBLER, REXX
Databases: DB2, VSAM
Online Systems: CICS
Tools: ChangeMan, Endeavor, File-AID, Abend-AID, Expeditor, Control-M, CA7, ServiceNow, Remedy
Methodologies: Agile (Scrum, Kanban), Waterfall, DevOps
Doma

In [ ]:
from google.colab import files
uploaded = files.upload()

print("\nUploaded files:")
for fn in uploaded.keys():
    print(fn)


Saving Bhim_Singh_Senior COBOL_Developer_Resume.docx to Bhim_Singh_Senior COBOL_Developer_Resume.docx

Uploaded files:
Bhim_Singh_Senior COBOL_Developer_Resume.docx


In [ ]:
# Chunking helper
def chunk_text(text, chunk_size=1200, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# Create chunks
chunks = chunk_text(raw_text, chunk_size=1200, overlap=200)
print(f"Total chunks created: {len(chunks)}")

# Convert chunks to LangChain Document objects
from langchain.schema import Document
documents = [Document(page_content=c, metadata={"source": file_path, "chunk_id": i})
             for i, c in enumerate(chunks)]

print("Sample chunk preview:\n")
print(documents[0].page_content[:500])


Total chunks created: 3
Sample chunk preview:

BHIM SINGH

📞 +91 9810811949 | ✉️ bhim.sb@gmail.com | 🌐 www.linkedin.com/in/bhim-singh-mainframelead

PROFILE SUMMARY

Senior Mainframe Developer with 13+ years of hands-on experience in designing, coding, testing, and maintaining large-scale Mainframe applications using COBOL, JCL, VSAM, DB2, and CICS across banking, insurance, and healthcare domains. Skilled in performance tuning, production support, batch optimization, and end-to-end SDLC delivery. Strong problem-solver with proven ability to


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load free embedding model
hf_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("HuggingFace model loaded!")


# LangChain-compatible embedding class (FIXED)
class HFEmbeddings:
    def embed_documents(self, texts):
        return hf_model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return hf_model.encode([text], convert_to_numpy=True)[0].tolist()

    # ⭐ FIX: make object callable
    def __call__(self, text):
        return self.embed_query(text)


embeddings = HFEmbeddings()

# Prepare texts & metadata
text_list = [d.page_content for d in documents]
metadata_list = [d.metadata for d in documents]

# Compute embeddings
print("Generating embeddings...")
text_embeddings = embeddings.embed_documents(text_list)
print("Generated", len(text_embeddings), "embeddings.")

# Pair text & vectors for FAISS
text_embedding_pairs = list(zip(text_list, text_embeddings))

# Build FAISS vectorstore
vectorstore = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,
    embedding=embeddings,       # FAISS will now call embeddings(text)
    metadatas=metadata_list
)

print("\nFAISS Index created successfully!")
print("Total vectors stored:", vectorstore.index.ntotal)


HuggingFace model loaded!
Generating embeddings...


Generated 3 embeddings.

FAISS Index created successfully!
Total vectors stored: 3


In [ ]:
!pip install -q sentence-transformers


In [ ]:
query = "Experience with COBOL, JCL, production support and banking payments"
hits = vectorstore.similarity_search(query, k=3)

for i, hit in enumerate(hits, 1):
    header = f"HIT {i} — source: {hit.metadata.get('source')} chunk_id: {hit.metadata.get('chunk_id')}"
    print(colored(header, "cyan"))
    print(hit.page_content[:800].strip() + ("..." if len(hit.page_content)>800 else ""))
    print()


HIT 1 — source: /content/Bhim_Singh_Senior COBOL_Developer_Resume.docx chunk_id: 1
Analyst

June 2021 – September 2025

• Developed and optimized COBOL and JCL modules for multiple batch and online systems, improving runtime efficiency by 20%.
• Provided end-to-end application support for critical banking and payment systems, ensuring 98% uptime.
• Debugged and resolved high-priority incidents using Abend-AID and File-AID, improving MTTR by 25%.
• Implemented preventive fixes and RCA-based improvements, reducing recurring job failures by 20%.
• Executed 15–20 year changes and deployments with 100% compliance to change management standards.

HCL TECHNOLOGIES — Senior Project Lead / Mainframe Developer

December 2016 – June 2021

• Designed, coded, and tested new COBOL programs and DB2 queries for batch and CICS-based systems.
• Automated repetitive JCL processes using RE...

HIT 2 — source: /content/Bhim_Singh_Senior COBOL_Developer_Resume.docx chunk_id: 2
g batch jobs to reduce executi

In [ ]:
import google.generativeai as genai

def answer_with_retrieval_gemini(question, k=3):
    hits = vectorstore.similarity_search(question, k=k)
    context = "\n\n---\n\n".join([h.page_content for h in hits])

    prompt = f"""
Use the following resume excerpts to answer the question.
If the answer is not found, reply exactly: "Not found in resume".

Context:
{context}

Question:
{question}

Answer:
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# Test
question = "What mainframe technologies does Bhim Singh have experience with?"
print("Q:", question)
print("\nA:", answer_with_retrieval_gemini(question))


Q: What mainframe technologies does Bhim Singh have experience with?

A: Bhim Singh has experience with the following mainframe technologies: COBOL, JCL, VSAM, DB2, CICS, PL/I, ASSEMBLER, REXX, ChangeMan, Endeavor, File-AID, Abend-AID, Expeditor, Control-M, CA7, ServiceNow, and Remedy.


In [ ]:
!pip install -q google-generativeai


In [ ]:
import google.generativeai as genai
from getpass import getpass

GEMINI_KEY = getpass("Enter your FREE Gemini API key: ")
genai.configure(api_key=GEMINI_KEY)


Enter your FREE Gemini API key: ··········


In [ ]:
for m in genai.list_models():
    print(m.name)


models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-l

In [ ]:
# If you want TimeWeightedVectorStoreRetriever or GenerativeAgent behavior:
print("Optional notes:")
print("- TimeWeightedVectorStoreRetriever API may have moved; if needed I can add a compatible example.")
print("- GenerativeAgent (experimental) imports were attempted earlier. If you want a working multi-agent example I can provide one, but it may require pinning specific langchain versions.")
